# Create ISA-API Investigation from Datascriptor Study Design configuration
# Crossover Study with radiological and drug treatment on Mice

In this notebook I will show you how you can use a study design configuration is JSON format as produce by datascriptor (https://gitlab.com/datascriptor/datascriptor) to generate a single-study ISA investigation and how you can then serialise it in JSON and tabular (i.e. CSV) format.

Or study design configuration consists of:
- a 4-arm study design. Each arm has 10 subjects
- subjects are mice. There is an observational factor, named "sex" with two values: "M" and "F"
- a crossover of two treatments, a drug treatment (cyclophosphamide 500 mg for 20 days) and a radiological treatment ("strontium chloride sr 89" 2 MBq/Kg forr 40 days)
- four non-treatment phases: screen, run-in, washout and follow-up
- three sample types colllected: tissue, blood, and urine
- three assay types: 
    - metabolite profiling through mass spectrometry on the urine sample.
    - clinical chemistry with marker on blood samples
    - genomic sequencing on tissue samples

## 1. Setup

Let's import all the required libraries

In [1]:
from time import time
import os
import json

In [2]:
## ISA-API related imports
from isatools.model import Investigation, Study

In [3]:
## ISA-API create mode related imports
from isatools.create.model import StudyDesign
from isatools.create.connectors import generate_study_design

# serializer from ISA Investigation to JSON
from isatools.isajson import ISAJSONEncoder

# ISA-Tab serialisation
from isatools import isatab

In [4]:
## ISA-API create mode related imports
from isatools.create import model
from isatools import isajson

## 2. Load the Study Design JSON configuration

First of all we load the study design configurator with all the specs defined above

In [5]:
with open(os.path.abspath(os.path.join(
    "config", "crossover-study-design-2treatments-mice.json"
)), "r") as config_file:
    study_design_config = json.load(config_file)
study_design_config

{'name': 'Test study',
 'description': 'This design describes a study with repeated treatments on the same arms. We will be collecting three samples and these will undergo three different assays.',
 'subjectType': 'Mouse',
 'subjectSize': 10,
 'designType': {'term': 'crossover design',
  'id': 'OBI:0500003',
  'iri': 'http://purl.obolibrary.org/obo/OBI_0500003',
  'label': 'Study subjects receive repeated treatments',
  'value': 'crossover'},
 'observationalFactors': [{'name': 'sex',
   'values': ['M', 'F'],
   'isQuantitative': False,
   'unit': None}],
 'subjectGroups': {'selected': [{'name': 'SubjectGroup_0',
    'type': 'Mouse',
    'characteristics': [{'name': 'sex',
      'value': 'M',
      'unit': None,
      'isQuantitative': False}]},
   {'name': 'SubjectGroup_1',
    'type': 'Mouse',
    'characteristics': [{'name': 'sex',
      'value': 'F',
      'unit': None,
      'isQuantitative': False}]}],
  'unselected': []},
 'treatmentPlan': {'screen': {'selected': True,
   'name':

## 3. Generate the ISA Study Design from the JSON configuration
To perform the conversion we just need to use the function `generate_isa_study_design()` (name possibly subject to change, should we drop the "isa" and "datascriptor" qualifiers?)

In [6]:
study_design = generate_study_design(study_design_config)
assert isinstance(study_design, StudyDesign)

## 4. Generate the ISA Study from the StudyDesign and embed it into an ISA Investigation

The `StudyDesign.generate_isa_study()` method returns the complete ISA-API `Study` object.

In [7]:
start = time()
study = study_design.generate_isa_study()
end = time()
print('The generation of the study design took {:.2f} s.'.format(end - start))
assert isinstance(study, Study)
investigation = Investigation(identifier='inv01', studies=[study])

The generation of the study design took 1.32 s.


## 5. Serialize and save the JSON representation of the generated ISA Investigation

In [8]:
start = time()
inv_json = json.dumps(investigation, cls=ISAJSONEncoder, sort_keys=True, indent=4, separators=(',', ': '))
end = time()
print('The JSON serialisation of the ISA investigation took {:.2f} s.'.format(end - start))

The JSON serialisation of the ISA investigation took 0.56 s.


In [9]:
directory = os.path.abspath(os.path.join('output', 'crossover-2treatments-mice'))
os.makedirs(directory, exist_ok=True)
with open(os.path.abspath(os.path.join(directory, 'isa-investigation-crossover-2treatments-mice.json')), 'w') as out_fp:
    json.dump(json.loads(inv_json), out_fp)

## 6. Dump the ISA Investigation to ISA-Tab

In [10]:
start = time()
isatab.dump(investigation, directory)
end = time()
print('The Tab serialisation of the ISA investigation took {:.2f} s.'.format(end - start))

The Tab serialisation of the ISA investigation took 25.29 s.


To use them on the notebook we can also dump the tables to pandas DataFrames, using the `dump_tables_to_dataframes` function rather than dump

In [11]:
dataframes = isatab.dump_tables_to_dataframes(investigation)

In [12]:
len(dataframes)

4

## 7. Check the correctness of the ISA-Tab DataFrames 

We have 1 study file and 2 assay files (one for MS and one for NMR). Let's check the names:

In [13]:
for key in dataframes.keys():
    display(key)

's_study_01.txt'

'a_AT8_genome-sequencing_nucleic-acid-sequencing.txt'

'a_AT11_clinical-chemistry_marker-panel.txt'

'a_AT1_metabolite-profiling_mass-spectrometry.txt'

### 7.1 Count of subjects and samples

We have 10 subjects in the each of the six arms for a total of 60 subjects. 5 blood samples per subject are collected (1 in treatment 1 phase, 1 in treatment, and 3 in the follow-up phase) for a total of 300 blood samples. These will undergo the NMR assay. We have 4 saliva samples per subject (1 during screen and 3 during follow-up) for a total of 240 saliva samples. These will undergo the "mass spcetrometry" assay.

In [14]:
study_frame = dataframes['s_study_01.txt']
count_arm0_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'GRP0' in el)])
count_arm1_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'GRP1' in el)])
count_arm2_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'GRP2' in el)])
count_arm3_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'GRP3' in el)])
print("There are {} samples in the GRP0 arm (i.e. group)".format(count_arm0_samples))
print("There are {} samples in the GRP1 arm (i.e. group)".format(count_arm1_samples))
print("There are {} samples in the GRP2 arm (i.e. group)".format(count_arm2_samples))
print("There are {} samples in the GRP3 arm (i.e. group)".format(count_arm3_samples))

There are 160 samples in the GRP0 arm (i.e. group)
There are 160 samples in the GRP1 arm (i.e. group)
There are 160 samples in the GRP2 arm (i.e. group)
There are 160 samples in the GRP3 arm (i.e. group)


In [20]:
study_frame

,Source Name,Characteristics[Study Subject],Characteristics[sex],Protocol REF,Parameter Value[run order],Parameter Value[study cell],Date,Performer,Sample Name,Characteristics[organism part],Factor Value[DURATION],Unit,Factor Value[AGENT],Factor Value[INTENSITY],Unit.1
0,GRP0_SBJ01,Mouse,M,sample collection,112,A0E5,2021-03-11,Bob,GRP0_SBJ01_A0E5_SMP-Urine-2,Urine,90,days,,,
1,GRP0_SBJ01,Mouse,M,sample collection,133,A0E5,2021-03-11,Bob,GRP0_SBJ01_A0E5_SMP-Blood-Sample-3,Blood Sample,90,days,,,
2,GRP0_SBJ01,Mouse,M,sample collection,001,A0E0,2021-03-11,Bob,GRP0_SBJ01_A0E0_SMP-Blood-Sample-1,Blood Sample,10,days,,,
3,GRP0_SBJ01,Mouse,M,sample collection,041,A0E2,2021-03-11,Bob,GRP0_SBJ01_A0E2_SMP-Blood-Sample-1,Blood Sample,40,days,strontium chloride sr 89,2.0,MBq/Kg
4,GRP0_SBJ01,Mouse,M,sample collection,111,A0E5,2021-03-11,Bob,GRP0_SBJ01_A0E5_SMP-Urine-1,Urine,90,days,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,GRP3_SBJ10,Mouse,F,sample collection,493,A3E0,2021-03-11,Bob,GRP3_SBJ10_A3E0_SMP-Urine-1,Urine,10,days,,,
636,GRP3_SBJ10,Mouse,F,sample collection,594,A3E5,2021-03-11,Bob,GRP3_SBJ10_A3E5_SMP-Urine-2,Urine,90,days,,,
637,GRP3_SBJ10,Mouse,F,sample collection,615,A3E5,2021-03-11,Bob,GRP3_SBJ10_A3E5_SMP-Blood-Sample-2,Blood Sample,90,days,,,
638,GRP3_SBJ10,Mouse,F,sample collection,494,A3E0,2021-03-11,Bob,GRP3_SBJ10_A3E0_SMP-Urine-2,Urine,10,days,,,


In [15]:
dataframes['a_AT1_metabolite-profiling_mass-spectrometry.txt']

,Sample Name,Protocol REF,Performer,Extract Name,Characteristics[extract type],Term Accession Number,Protocol REF.1,Performer.1,Labeled Extract Name,Label,Protocol REF.2,Parameter Value[instrument],Term Accession Number.1,Parameter Value[injection_mode],Term Accession Number.2,Parameter Value[acquisition_mode],Term Accession Number.3,MS Assay Name,Performer.2,Raw Spectral Data File
0,GRP0_SBJ01_A0E0_SMP-Urine-1,extraction,Unknown,AT1-S1-Extract-R1,polar fraction,polar fraction,labeling,Unknown,AT1-S1-LE-R1,label_0,mass spectrometry,Agilent QTQF 6510,http://purl.obolibrary.org/obo/MS_1000676,LC,,positive mode,http://purl.obolibrary.org/obo/MS_1002807,AT1-S1-mass-spectrometry-Acquisition-R4,Unknown,AT1-S1-raw-spectral-data-file-R4.raw
1,GRP0_SBJ01_A0E0_SMP-Urine-1,extraction,Unknown,AT1-S1-Extract-R1,polar fraction,polar fraction,labeling,Unknown,AT1-S1-LE-R1,label_0,mass spectrometry,Agilent QTQF 6510,http://purl.obolibrary.org/obo/MS_1000676,FIA,http://purl.obolibrary.org/obo/MS_1000058,positive mode,http://purl.obolibrary.org/obo/MS_1002807,AT1-S1-mass-spectrometry-Acquisition-R2,Unknown,AT1-S1-raw-spectral-data-file-R2.raw
2,GRP0_SBJ01_A0E0_SMP-Urine-1,extraction,Unknown,AT1-S1-Extract-R1,polar fraction,polar fraction,labeling,Unknown,AT1-S1-LE-R1,label_0,mass spectrometry,Agilent QTQF 6510,http://purl.obolibrary.org/obo/MS_1000676,LC,,positive mode,http://purl.obolibrary.org/obo/MS_1002807,AT1-S1-mass-spectrometry-Acquisition-R3,Unknown,AT1-S1-raw-spectral-data-file-R3.raw
3,GRP0_SBJ01_A0E0_SMP-Urine-1,extraction,Unknown,AT1-S1-Extract-R1,polar fraction,polar fraction,labeling,Unknown,AT1-S1-LE-R1,label_0,mass spectrometry,Agilent QTQF 6510,http://purl.obolibrary.org/obo/MS_1000676,FIA,http://purl.obolibrary.org/obo/MS_1000058,positive mode,http://purl.obolibrary.org/obo/MS_1002807,AT1-S1-mass-spectrometry-Acquisition-R1,Unknown,AT1-S1-raw-spectral-data-file-R1.raw
4,GRP0_SBJ01_A0E0_SMP-Urine-2,extraction,Unknown,AT1-S2-Extract-R1,polar fraction,polar fraction,labeling,Unknown,AT1-S2-LE-R1,label_0,mass spectrometry,Agilent QTQF 6510,http://purl.obolibrary.org/obo/MS_1000676,FIA,http://purl.obolibrary.org/obo/MS_1000058,positive mode,http://purl.obolibrary.org/obo/MS_1002807,AT1-S2-mass-spectrometry-Acquisition-R2,Unknown,AT1-S2-raw-spectral-data-file-R2.raw
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1115,GRP3_SBJ10_A3E5_SMP-Urine-1,extraction,Unknown,AT1-S263-Extract-R1,polar fraction,polar fraction,labeling,Unknown,AT1-S263-LE-R1,label_0,mass spectrometry,Agilent QTQF 6510,http://purl.obolibrary.org/obo/MS_1000676,LC,,positive mode,http://purl.obolibrary.org/obo/MS_1002807,AT1-S263-mass-spectrometry-Acquisition-R3,Unknown,AT1-S263-raw-spectral-data-file-R3.raw
1116,GRP3_SBJ10_A3E5_SMP-Urine-2,extraction,Unknown,AT1-S264-Extract-R1,polar fraction,polar fraction,labeling,Unknown,AT1-S264-LE-R1,label_0,mass spectrometry,Agilent QTQF 6510,http://purl.obolibrary.org/obo/MS_1000676,FIA,http://purl.obolibrary.org/obo/MS_1000058,positive mode,http://purl.obolibrary.org/obo/MS_1002807,AT1-S264-mass-spectrometry-Acquisition-R2,Unknown,AT1-S264-raw-spectral-data-file-R2.raw
1117,GRP3_SBJ10_A3E5_SMP-Urine-2,extraction,Unknown,AT1-S264-Extract-R1,polar fraction,polar fraction,labeling,Unknown,AT1-S264-LE-R1,label_0,mass spectrometry,Agilent QTQF 6510,http://purl.obolibrary.org/obo/MS_1000676,LC,,positive mode,http://purl.obolibrary.org/obo/MS_1002807,AT1-S264-mass-spectrometry-Acquisition-R4,Unknown,AT1-S264-raw-spectral-data-file-R4.raw
1118,GRP3_SBJ10_A3E5_SMP-Urine-2,extraction,Unknown,AT1-S264-Extract-R1,polar fraction,polar fraction,labeling,Unknown,AT1-S264-LE-R1,label_0,mass spectrometry,Agilent QTQF 6510,http://purl.obolibrary.org/obo/MS_1000676,FIA,http://purl.obolibrary.org/obo/MS_1000058,positive mode,http://purl.obolibrary.org/obo/MS_1002807,AT1-S264-mass-spectrometry-Acquisition-R1,Unknown,AT1-S264-raw-spectral-data-file-R1.raw


In [16]:
dataframes['a_AT11_clinical-chemistry_marker-panel.txt']

,Sample Name,Protocol REF,Performer,Raw Data File
0,GRP0_SBJ01_A0E0_SMP-Blood-Sample-1,sample preparation,Unknown,AT11-S2-raw_data_file-R1
1,GRP0_SBJ01_A0E0_SMP-Blood-Sample-1,sample preparation,Unknown,AT11-S3-raw_data_file-R1
2,GRP0_SBJ01_A0E0_SMP-Blood-Sample-1,sample preparation,Unknown,AT11-S1-raw_data_file-R1
3,GRP0_SBJ01_A0E2_SMP-Blood-Sample-1,sample preparation,Unknown,AT11-S31-raw_data_file-R1
4,GRP0_SBJ01_A0E2_SMP-Blood-Sample-1,sample preparation,Unknown,AT11-S32-raw_data_file-R1
...,...,...,...,...
955,GRP3_SBJ10_A3E5_SMP-Blood-Sample-2,sample preparation,Unknown,AT11-S884-raw_data_file-R1
956,GRP3_SBJ10_A3E5_SMP-Blood-Sample-2,sample preparation,Unknown,AT11-S883-raw_data_file-R1
957,GRP3_SBJ10_A3E5_SMP-Blood-Sample-3,sample preparation,Unknown,AT11-S888-raw_data_file-R1
958,GRP3_SBJ10_A3E5_SMP-Blood-Sample-3,sample preparation,Unknown,AT11-S886-raw_data_file-R1


In [17]:
dataframes['a_AT8_genome-sequencing_nucleic-acid-sequencing.txt']

,Sample Name,Protocol REF,Performer,Extract Name,Characteristics[extract type],Protocol REF.1,Parameter Value[instrument],Parameter Value[library_orientation],Parameter Value[library_strategy],Performer.1,Raw Data File
0,GRP0_SBJ01_A0E4_SMP-Tissue-Sample-1,extraction,Unknown,AT8-S2-Extract-R2,gDNA,library_preparation,Illumina NovaSeq 6000,single,WGS,Unknown,AT8-S2-raw_data_file-R5.raw
1,GRP0_SBJ01_A0E4_SMP-Tissue-Sample-1,extraction,Unknown,AT8-S1-Extract-R2,gDNA,library_preparation,Illumina NovaSeq 6000,single,AMPLICON,Unknown,AT8-S1-raw_data_file-R8.raw
2,GRP0_SBJ01_A0E4_SMP-Tissue-Sample-1,extraction,Unknown,AT8-S2-Extract-R2,gDNA,library_preparation,Illumina NovaSeq 6000,single,WGS,Unknown,AT8-S2-raw_data_file-R6.raw
3,GRP0_SBJ01_A0E4_SMP-Tissue-Sample-1,extraction,Unknown,AT8-S1-Extract-R2,gDNA,library_preparation,Illumina NovaSeq 6000,single,WGS,Unknown,AT8-S1-raw_data_file-R6.raw
4,GRP0_SBJ01_A0E4_SMP-Tissue-Sample-1,extraction,Unknown,AT8-S1-Extract-R2,gDNA,library_preparation,Illumina NovaSeq 6000,single,AMPLICON,Unknown,AT8-S1-raw_data_file-R7.raw
...,...,...,...,...,...,...,...,...,...,...,...
635,GRP3_SBJ10_A3E2_SMP-Tissue-Sample-1,extraction,Unknown,AT8-S64-Extract-R1,DNA,library_preparation,Illumina NovaSeq 6000,single,AMPLICON,Unknown,AT8-S64-raw_data_file-R2.raw
636,GRP3_SBJ10_A3E2_SMP-Tissue-Sample-1,extraction,Unknown,AT8-S64-Extract-R1,DNA,library_preparation,Illumina NovaSeq 6000,single,WGS,Unknown,AT8-S64-raw_data_file-R3.raw
637,GRP3_SBJ10_A3E2_SMP-Tissue-Sample-1,extraction,Unknown,AT8-S64-Extract-R1,DNA,library_preparation,Illumina NovaSeq 6000,single,AMPLICON,Unknown,AT8-S64-raw_data_file-R1.raw
638,GRP3_SBJ10_A3E2_SMP-Tissue-Sample-1,extraction,Unknown,AT8-S63-Extract-R2,gDNA,library_preparation,Illumina NovaSeq 6000,single,WGS,Unknown,AT8-S63-raw_data_file-R6.raw


# 7.1 Overview of the Mass Spec assay table

For the mass. spec. assay table, we have 160 (derma) samples, 160 extracts (1 per  sample, "polar" fraction), 320 labeled extracts (2 per extract, as "#replicates" is  2) and 640 mass spec protocols + 640 output files (2 per labeled extract as we do 1 technical replicate with 2 protocol parameter combinations `["Agilent QTQF 6510", "FIA", "positive mode"]` and `["Agilent QTQF 6510", "LC", "positive mode"]`).

In [18]:
dataframes['a_AT1_metabolite-profiling_mass-spectrometry.txt'].nunique(axis=0, dropna=True)

Sample Name                           280
Protocol REF                            1
Performer                               1
Extract Name                          280
Characteristics[extract type]           1
Term Accession Number                   1
Protocol REF.1                          1
Performer.1                             1
Labeled Extract Name                  280
Label                                   1
Protocol REF.2                          1
Parameter Value[instrument]             1
Term Accession Number.1                 1
Parameter Value[injection_mode]         2
Term Accession Number.2                 2
Parameter Value[acquisition_mode]       1
Term Accession Number.3                 1
MS Assay Name                        1120
Performer.2                             1
Raw Spectral Data File               1120
dtype: int64

### Overview of the Genome Sequencing assay table

For the NMR assay table, we have 360 (blood) samples, 720 extracts (2 per  sample, a single replicate of the "supernatant" and "pellet" fractions) and 2880 NMR protocols + 4800 output files (4 per extract as we do 2 technical replicates with 2 protocol parameter combinations `["Bruker Avance II 1 GHz", "1D 1H NMR", "CPGM"]` and `["Bruker Avance II 1 GHz", "1D 1H NMR", "TOCSY"]`).

In [19]:
dataframes['a_AT8_genome-sequencing_nucleic-acid-sequencing.txt'].nunique(axis=0, dropna=True)

Sample Name                              40
Protocol REF                              1
Performer                                 1
Extract Name                            160
Characteristics[extract type]             2
Protocol REF.1                            1
Parameter Value[instrument]               1
Parameter Value[library_orientation]      1
Parameter Value[library_strategy]         2
Performer.1                               1
Raw Data File                           640
dtype: int64